In [1]:
%matplotlib qt
import os
import config as cfg
import hyperspy.api as hs
from skimage import io
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import envi_header
from sklearn.cluster import SpectralClustering

image_id = 'frt00003bfb'

# read header file
hdr_filename = image_id + '_07_if166j_mtr3.hdr'
hdr_filepath = os.path.join(cfg.DATA_DIR, hdr_filename)
header = envi_header.read_hdr_file(hdr_filepath)

print('Imported {}'.format(hdr_filename))
print('wavelength units - {}'.format(header['wavelength units']))

# read tiff file
img_filename = image_id + '_data_cube.tif'
img_filepath = os.path.join(cfg.DATA_DIR, img_filename)
img = io.imread(img_filepath)

print('Imported {}'.format(img_filename))

c, w, h = img.shape
img = img.transpose(1, 2, 0)

# remove ignored values
img[img == header['data ignore value']] = 0

Imported frt00003bfb_07_if166j_mtr3.hdr
wavelength units - Nanometers


/Users/maxhipperson/.virtualenvs/marsnet-ZEfWMKGE/lib/python3.6/site-packages/skimage/external/tifffile/tifffile.py:2128: UserWarning: tags are not ordered by code
  warnings.warn("tags are not ordered by code")


Imported frt00003bfb_data_cube.tif


In [9]:
'''
Import the image into a signal class object
'''

axes_x = {'name': 'x', 'size': img.shape[0], 'units': 'px'}
axes_y = {'name': 'y', 'size': img.shape[1], 'units': 'px'}
axes_w = {'name': 'wavelength band', 'size': img.shape[2], 'units': 'index'}

# convert image to signal object
im = hs.signals.Signal1D(img, axes=[axes_x, axes_y, axes_w])
im.axes_manager

# Set wavelength bounds to crop the spectrum to.
crop_spectra = True

if crop_spectra:
    
    lower = 1000
    upper = 2800
    
    # find the index of the boundary wavelengths in the header
    wavelength = np.array(header['wavelength'])
    lower_index = np.argmax(wavelength >= lower)
    upper_index = np.argmax(wavelength > upper) - 1
    
    # crop the signal to the specified range
    im.crop_signal1D(lower_index, upper_index)
    
    # crop to central section of image
    # im.crop('x', 200, 600)
    # im.crop('y', 200, 600)

im.plot()

In [10]:
im.decomposition()
im.learning_results.summary()
im.plot_explained_variance_ratio(threshold=0.005)
im.plot_decomposition_results()

Decomposition parameters:
-------------------------

Decomposition algorithm : 	svd
Poissonian noise normalization : False
Output dimension : None
Centre : None


In [7]:
factors = im.get_decomposition_factors()
fig = plt.figure(figsize=(10, 10))
hs.plot.plot_spectra(factors.inav[:9], legend='auto', fig=fig)

In [12]:
loadings = im.get_decomposition_loadings()
fig = plt.figure(figsize=(15, 10))
hs.plot.plot_images(loadings.inav[:9], tight_layout=True, fig=fig)

In [24]:
test = im.get_decomposition_loadings()
test.plot()


In [9]:
imc = im.get_decomposition_model(components=6)

# calculate and display residuals
(im - imc).plot()

In [11]:
# independent componant analysis
im.blind_source_separation(number_of_components=20)
im.learning_results.summary()

/Users/maxhipperson/.virtualenvs/marsnet-ZEfWMKGE/lib/python3.6/site-packages/sklearn/decomposition/fastica_.py:121: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)


Decomposition parameters:
-------------------------

Decomposition algorithm : 	svd
Poissonian noise normalization : False
Output dimension : None
Centre : None

Demixing parameters:
------------------------
BSS algorithm : sklearn_fasticaNumber of components : 20


In [12]:
im.plot_bss_results()

In [ ]:
data = im.data
old_shape = data.shape
print(old_shape)

n_clusters = 8

# get clusters

# flatten the data
flat = np.reshape(data, (old_shape[0]*old_shape[1], old_shape[2]))
print(flat.shape)

# for i in range(old_shape[0]):
#     y = i
#     for j in range(old_shape[1]):
#         x = j
#         
#         old_spectrum = data[i, j, :]
#         new_spectrum = flat[y * old_shape[0] + x, :]
#         
#         issame = np.array_equal(old_spectrum, new_spectrum)
#         if not issame:
#             print('AHHHH')

# normalise each spectrum
# print(flat.mean(axis=1, keepdims=True).shape)
# print(flat.mean(axis=1, keepdims=True)[:3])
# print(flat.std(axis=1, keepdims=True)[:3])
#       
flat -= flat.mean(axis=1, keepdims=True)
flat /= flat.std(axis=1, keepdims=True)
# 
# print(flat.mean(axis=1, keepdims=True)[:3])
# print(flat.std(axis=1, keepdims=True)[:3])

sc = SpectralClustering(n_clusters=n_clusters,
                        # assign_labels='discretize',
                        ).fit(flat)
labels = sc.labels_
print(labels.shape)

NameError: name 'im' is not defined

In [ ]:

sc.fit_predict()

AttributeError: module 'sklearn' has no attribute 'cluster'